In [ ]:
from experiment_project.utils.initial.util import init_sys_env
from experiment_project.utils.files.read import read_yaml
import dspy
from dspy.teleprompt import BootstrapFewShot

from dspy.datasets.gsm8k import GSM8K, gsm8k_metric
init_sys_env()
secret_env_file = '/mnt/d/project/dy/extra/autogen/env_secret_config.yaml'

api_configs = read_yaml(secret_env_file)
model_config = api_configs.get('openai')
turbo = dspy.OpenAI(model=model_config.get('model'), max_tokens=2048,api_key=model_config.get('api_key'))
dspy.settings.configure(lm=turbo)


In [2]:
class BasicQA(dspy.Signature):
  """回答问题，答案要简短。"""
  question = dspy.InputField()
  answer = dspy.OutputField(desc="通常是 1 到 5 个词", prefix="问题的答案:") 
  

In [8]:
predictor = dspy.Predict(BasicQA)
result = predictor.forward(question="1+1=?")
result.answer

In [5]:
turbo.history[-1]

In [7]:
turbo.inspect_history(n=1)

In [17]:


class Writer(dspy.Signature):
  question = dspy.InputField()
  answer = dspy.OutputField(desc="通常是一段话,并且详细的说明为什么?")
  tone = dspy.OutputField(desc="使用鲁迅先生的语气和语法回答这段话")
  

In [18]:
writer_predictor = dspy.Predict(Writer)
writer_result = writer_predictor.forward(question="如何评价阿Q精神")
writer_result.answer

In [21]:
turbo.history[-1]

In [20]:
turbo.inspect_history(n=1)
